In [31]:
from pathlib import Path
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
load_dotenv()
base_path = Path.home() / Path(os.environ.get("DATA_PATH"))
print(base_path)

/home/aayush/rsys_data


In [32]:
news = pd.read_csv(
    base_path/ Path("MINDlarge_train/news.tsv"), 
    sep="\t",
    names=["itemId","category","subcategory","title","abstract","url","title_entities","abstract_entities"])
print(f"The article data consist in total of {len(news)} number of articles.")
news

The article data consist in total of 101527 number of articles.


,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N45436,news,newsscienceandtechnology,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...,https://assets.msn.com/labs/mind/AABmf2I.html,"[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ..."
2,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
3,N86255,health,medical,Dispose of unwanted prescription drugs during ...,NaN,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[]
4,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
...,...,...,...,...,...,...,...,...
101522,N115249,sports,more_sports,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b...",https://assets.msn.com/labs/mind/BBWzQnK.html,[],[]
101523,N64337,finance,finance-real-estate,Mansion Monday: Contemporary Des Moines home i...,Among the perks of this unique Des Moines home...,https://assets.msn.com/labs/mind/BBWzQq8.html,"[{""Label"": ""Des Moines, Iowa"", ""Type"": ""G"", ""W...","[{""Label"": ""Des Moines, Iowa"", ""Type"": ""G"", ""W..."
101524,N100102,sports,soccer_epl,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they ...",https://assets.msn.com/labs/mind/BBWzQuK.html,"[{""Label"": ""MLS Cup"", ""Type"": ""U"", ""WikidataId...",[]
101525,N74617,autos,autossports,Best Sports Car Deals for October,NaN,https://assets.msn.com/labs/mind/BBy5rVe.html,"[{""Label"": ""Peugeot RCZ"", ""Type"": ""V"", ""Wikida...",[]


In [33]:
c=news[['category','subcategory']].value_counts()
index=[]
for i in c.index:
    index.append(np.array(i))
index=np.array(index)
df=pd.DataFrame(columns=['category','subcategory','values'])
df['category']=index[:,0]
df['subcategory']=index[:,1]
df['values']=c.values
px.bar(data_frame=df,x='category',y='values',color='subcategory')

In [34]:
print('the number of articles before processing :',len(news))
news.drop_duplicates(subset=['title'],inplace=True)
print('The number of articles after processing :',len(news))

the number of articles before processing : 101527
The number of articles after processing : 98388


In [35]:
file_path_entity = base_path / Path("MINDlarge_train/entity_embedding.vec")
file_path_relations = base_path / Path("MINDlarge_train/relation_embedding.vec")
def load_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, encoding="utf-8") as file:
        for line in file:
            values = line.strip().split()
            entity_id = values[0]
            embedding = np.array(values[1:], dtype="float32")
            embeddings_index[entity_id] = embedding
    return embeddings_index
entity_embeddings=load_embeddings(file_path_entity)
key_list = list(entity_embeddings.keys())
print('The number of entities :',len(key_list))

The number of entities : 42007


In [36]:
relations_embeddings=load_embeddings(file_path_relations)
relation_key_list = list(relations_embeddings.keys())
print('The number of entities :',len(relation_key_list))

The number of entities : 1091


In [37]:
sub_category=news['subcategory'].nunique()
category=news['category'].nunique()
sub_category

284

In [38]:
categories = news['category'].unique()

# Create a function to convert categories to a list of 1s and 0s
def category_to_list(cat):
    return [1 if cat == category else 0 for category in categories]

# Apply the function to create the 'category_list' column
news['category_list'] = news['category'].apply(category_to_list)

In [39]:
news

,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities,category_list
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,N45436,news,newsscienceandtechnology,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...,https://assets.msn.com/labs/mind/AABmf2I.html,"[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,N86255,health,medical,Dispose of unwanted prescription drugs during ...,NaN,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[],"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...
101522,N115249,sports,more_sports,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b...",https://assets.msn.com/labs/mind/BBWzQnK.html,[],[],"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
101523,N64337,finance,finance-real-estate,Mansion Monday: Contemporary Des Moines home i...,Among the perks of this unique Des Moines home...,https://assets.msn.com/labs/mind/BBWzQq8.html,"[{""Label"": ""Des Moines, Iowa"", ""Type"": ""G"", ""W...","[{""Label"": ""Des Moines, Iowa"", ""Type"": ""G"", ""W...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
101524,N100102,sports,soccer_epl,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they ...",https://assets.msn.com/labs/mind/BBWzQuK.html,"[{""Label"": ""MLS Cup"", ""Type"": ""U"", ""WikidataId...",[],"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
101525,N74617,autos,autossports,Best Sports Car Deals for October,NaN,https://assets.msn.com/labs/mind/BBy5rVe.html,"[{""Label"": ""Peugeot RCZ"", ""Type"": ""V"", ""Wikida...",[],"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [40]:
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

# Replace 'path_to_glove_file' with the actual path to your GloVe file
glove_path = Path.home() / Path(os.environ.get("DATA_PATH")/Path('glove.6B.100d.txt'))
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file=glove_path, word2vec_output_file="gensim_glove_vectors.txt")
glove_model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)


/tmp/ipykernel_3837237/1022473072.py:7: DeprecationWarning:

Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).



In [41]:
from nltk.tokenize import word_tokenize
import nltk

# Download NLTK resources
nltk.download('punkt')
def get_item_embedding(tokens, embedding_model):
    # Filter tokens that have embeddings in the model
    valid_tokens = [token for token in tokens if token in embedding_model]
    
    # Check if there are valid tokens
    if not valid_tokens:
        return None
    
    # Calculate the mean of the embeddings
    embedding = sum(embedding_model[token] for token in valid_tokens) / len(valid_tokens)
    return embedding

[nltk_data] Downloading package punkt to /home/aayush/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [42]:
news['embedding'] = news.apply(lambda row: get_item_embedding(word_tokenize(str(row['title']) + ' ' + str(row['abstract'])), glove_model), axis=1)

In [43]:
news

,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities,category_list,embedding
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.12538737, 0.18832362, 0.46491063, -0.54959..."
1,N45436,news,newsscienceandtechnology,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...,https://assets.msn.com/labs/mind/AABmf2I.html,"[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.046219237, 0.09747844, 0.48073766, -0.31746..."
2,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.07985605, 0.21864629, 0.35760373, -0.33946..."
3,N86255,health,medical,Dispose of unwanted prescription drugs during ...,NaN,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[],"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.20300163, -0.103868246, 0.3819184, -0.07974..."
4,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.16571806, 0.067524046, 0.33379403, -0.2151..."
...,...,...,...,...,...,...,...,...,...,...
101522,N115249,sports,more_sports,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b...",https://assets.msn.com/labs/mind/BBWzQnK.html,[],[],"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-4.874079e-05, 0.21773942, 0.43500683, -0.392..."
101523,N64337,finance,finance-real-estate,Mansion Monday: Contemporary Des Moines home i...,Among the perks of this unique Des Moines home...,https://assets.msn.com/labs/mind/BBWzQq8.html,"[{""Label"": ""Des Moines, Iowa"", ""Type"": ""G"", ""W...","[{""Label"": ""Des Moines, Iowa"", ""Type"": ""G"", ""W...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[-0.08978617, 0.18038589, 0.52556604, -0.09335..."
101524,N100102,sports,soccer_epl,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they ...",https://assets.msn.com/labs/mind/BBWzQuK.html,"[{""Label"": ""MLS Cup"", ""Type"": ""U"", ""WikidataId...",[],"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.02630369, 0.13129476, 0.24318646, -0.60029..."
101525,N74617,autos,autossports,Best Sports Car Deals for October,NaN,https://assets.msn.com/labs/mind/BBy5rVe.html,"[{""Label"": ""Peugeot RCZ"", ""Type"": ""V"", ""Wikida...",[],"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[-0.0921965, 0.05617, 0.099253505, -0.08494049..."


In [44]:
feather_path=base_path/ Path("MINDlarge_train/news_glove_embed.feather")
news.to_feather(feather_path)

In [45]:

embedding_dict = dict(zip(news['itemId'], news['embedding']))




In [30]:
embedding_dict

{'N88753': array([-0.12538737,  0.18832362,  0.46491063, -0.54959106,  0.0262017 ,
         0.19335762, -0.04730933,  0.22277443, -0.10193846, -0.01146178,
         0.18939917, -0.2561144 ,  0.22104654,  0.18383099,  0.21855271,
        -0.38657808,  0.17056781,  0.09261394, -0.37655252,  0.14584023,
         0.24837184, -0.02282205,  0.46510595,  0.19478823,  0.39461854,
         0.0200286 , -0.12431882, -0.55322504,  0.02377389, -0.20676339,
        -0.07055233,  0.36227587, -0.2255266 ,  0.08308028, -0.05201406,
         0.23561369,  0.20501876,  0.35642746,  0.00530533, -0.23262705,
        -0.1784625 , -0.35096937,  0.04159489, -0.38031328, -0.24926768,
        -0.17970267,  0.36887246, -0.42946446,  0.06474356, -0.62449074,
        -0.0501981 , -0.04680388, -0.01035277,  1.0384289 , -0.25606713,
        -2.2447505 ,  0.04588058, -0.14250535,  1.3376117 ,  0.7319485 ,
        -0.28910446,  0.67500746, -0.18908045,  0.1403277 ,  0.74729395,
        -0.10901456,  0.66385216,  0.3732